# RAG pipeline 구축하기 1


Rag pipeline을 만들고 각 함수를 손쉽게 사용해보자

이번 포스팅에는 Retrieval를 통해 VectorDB의 정보를 잘 검색하여 가져오는지 확인해 본다.


## 0. Setting 

.env 파일을 만들어 API키들을 넣어준다. 

나중에 Ollema를 사용해서 API 없이 local에서 작동가능한 LLM을 사용해 보자.



```bash
OPENAI_API_KEY='sk-proj-5m5haMMQ0Sgkctb7Udixxxxx'


```

In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv
# API KEY 정보로드
load_dotenv()

True

## 1. RAG pipeline - Retrieval

In [2]:
# langchain_openai에서 ChatOpenAI(LLM)과 OpenAIEmbeddings(임베딩모델: text를 vector화하는 모델)을 load
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# PDF파일등 데이터를 Chroma형식의 vectorDB에 저장하고 리트리버가 수집한 데이터에 접근하기 위해 Chroma를 load
from langchain_community.vectorstores import Chroma

# 우리가 만든 config.py에서 모델등 옵션들을 수정
from utils.config import config, metadata_field_info

# 추가 데이터를 업로드하기 위해 file을 다큐먼트로 만들고 (convert_file_to_documents) 다큐먼트를 자르는 (split_document) 함수를 load합니다.
from utils.update import convert_file_to_documents

In [3]:
class Ragpipeline:
    def __init__(self):
        # chatGPT API를 통해 llm 모델 로드
        self.llm = ChatOpenAI(
            model=config["llm_predictor"]["model_name"],  # chatgpt 모델 이름
            temperature=config["llm_predictor"]["temperature"],  # 창의성 0~1
        )
        
        # 초기화 리스트들 
        
        # 1. RAG가 접근할 vectorDB를 초기화합니다.
        self.vector_store   = self.init_vectorDB()
        # 2. LLM이 질문에 대한 답변을 생성하기 전 질문에 관련된 컨텐츠 기반 답변을 생성하기 위해, 컨텐츠를 검색해 찾는 리트리버를 초기화 합니다. 
        self.retriever      = self.init_retriever()  

    def init_vectorDB(self, persist_directory=config["chroma"]["persist_dir"]):
        """vectorDB 설정"""
        embeddings = OpenAIEmbeddings(model=config["embed_model"]["model_name"])  # VectorDB에 저장될 데이터를 임베딩할 모델을 선언합니다.
        vector_store = Chroma(
            persist_directory=persist_directory,  # 기존에 vectordb가 있으면 해당 위치의 vectordb를 load하고 없으면 새로 생성합니다.
            embedding_function=embeddings,                      # 새롭게 데이터가 vectordb에 넣어질때 사용할 임베딩 방식을 정합니다, 저희는 위에서 선언한 embeddings를 사용합니다.
            collection_name = 'india',                          # india라는 이름을 정해줌으로써 나중에 vector store 관리 가능 
            collection_metadata = {'hnsw:space': 'cosine'},     # cosine 말고 l2 가 default / collection_metadata를 통해 유사도 검색에 사용될 공간('hnsw:space')을 'cosine'으로 지정하여, 코사인 유사도를 사용
        )
        return vector_store

    def init_retriever(self):
        """ Retriever 초기화 """               
        # base retriever 1         
        # retriever = self.vector_store.as_retriever(
        #     search_type   = "similarity",                       # similarity, score_threshold, mmr 
        #     search_kwargs = {"k": config["retriever_k"]},
            
        # )
        
        # base retriever 2 
        # retriever = self.vector_store.as_retriever(
        #     search_type="similarity_score_threshold",
        #     search_kwargs={"score_threshold": 0.75,
        #                    "k": config["retriever_k"]}
        # )
        
        # base retriever 3 
        retriever = self.vector_store.as_retriever(
            search_type="mmr",                                              # mmr 검색 방법으로 
            search_kwargs={'fetch_k': 10, "k": 5, 'lambda_mult': 0.4},      # 상위 10개의 관련 context에서 최종 5개를 추리고 'lambda_mult'는 관련성과 다양성 사이의 균형을 조정하는 파라메타 default 값이 0.5
            
        )
        
        return retriever
            

In [4]:
pipeline = Ragpipeline()

In [5]:
base_retriever = pipeline.init_retriever()

In [6]:
query = '인도 통관 및 운송'   # 질문할 문장

In [7]:
result = base_retriever.get_relevant_documents(query)

c:\Users\kim_h\anaconda3\envs\bigproject\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [8]:
for i, doc in enumerate(result):
    print(i)
    print(f"문서 내용: {doc.page_content}") # 문서 내용 표시
    print('---'*10)

0
문서 내용: 5. 통관 및 운송
 
 
가. 통관제도
  
 
통관 유형별 절차
 
1) 정식통관 
 
인도에서 일반적인 경우 통관에 소요되는 시간은 행정상 운송 수입의 경우 3~4 근무일, 항공 운송은 약 1~2 근무일이 소요된다. 일단 물
품이 입항하게 되면 보세구역장치에 적하되고, 이후 전자데이터 교환(EDI) 신고 또는 수작업신고 통해 수입신고를 하게 된다. 이후 인도
관세청의 수입요건, 관세 평가 등을 검사하는 Appraiser Section과 관련 서류 및 관세율을 재점검하는 Audit Section 과정을 거치게 되
며, 이를 통과하면 모든 서류에 서명 받게 된다. 이후 Challan No.라는 인도 관세 납부를 위한 번호를 부여받게 되고, 이를 근거로 관세를
완납하게 되면 물품을 반출할 수 있다. 또한, 정식통관의 경우 인도 세관은 일반적으로 화물 도착 후 7일 이후부터 체화료(Demurrage
Charge)를 부과하고, 컨테이너 운송의 경우 5일 이후부터 항공운송인 경우 3일 이
------------------------------
1
문서 내용: 도 증빙이 충분치 않다는 이유로 통관을 거부하는 사례도 자주 발생한다.  
 
  ㅇ 인도 지역별 세관관할: 인도의 세관 행정은 명목상 통일되어 있으나, 지역별 차이나 관할 세관 당국 간 소통 미비로 인하여 관세통관
상의 애로 해소가 발생하는 경우가 있다. 사전품목분류 심사제도의 경우에도 각 관할 세관별로 운영할 가능성이 높으며, 이에 한 관할지에
서 사전품목분류를 확정받았다 하더라도 다른 관할지에서는 이를 그대로 적용하지 않을 가능성이 있다.  
 
  ㅇ 서류의 수정: 인도는 통관에 필요한 서류에 기재 오류나 상호 간의 불일치가 있을 시 이를 수정하기가 매우 어려우며 이로 인해 통관
절차가 2, 3주씩 지연되기도 한다. 영문 한 글자 오타를 수정하는 데 3주를 허비해야 할 수도 있으며, 허위신고를 이유로 과태료가 발생할
수 있다.   
 
 
나. 주요 포워딩 업체 및 통관업체
 
 
ㅇ K

## Multi-Query와 Self-Query retriever 추가하기

In [9]:
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.retrievers.self_query.base import SelfQueryRetriever

In [10]:
class Ragpipeline:
    def __init__(self):
        # chatGPT API를 통해 llm 모델 로드
        self.llm = ChatOpenAI(
            model=config["llm_predictor"]["model_name"],  # chatgpt 모델 이름
            temperature=config["llm_predictor"]["temperature"],  # 창의성 0~1
        )
        
        # 초기화 리스트들 
        
        self.vector_store   = self.init_vectorDB()                  # 1. RAG가 접근할 vectorDB를 초기화합니다.
        self.retriever      = self.init_retriever()                 # 2. LLM이 질문에 대한 답변을 생성하기 전 질문에 관련된 컨텐츠 기반 답변을 생성하기 위해, 컨텐츠를 검색해 찾는 리트리버를 초기화 합니다. 
        self.mq_retriever   = self.init_multi_query_retriever()
        self.sq_retriever   = self.init_self_query_retriever()

    def init_vectorDB(self, persist_directory=config["chroma"]["persist_dir"]):
        """vectorDB 설정"""
        embeddings = OpenAIEmbeddings(model=config["embed_model"]["model_name"])  # VectorDB에 저장될 데이터를 임베딩할 모델을 선언합니다.
        vector_store = Chroma(
            persist_directory=persist_directory,  # 기존에 vectordb가 있으면 해당 위치의 vectordb를 load하고 없으면 새로 생성합니다.
            embedding_function=embeddings,                      # 새롭게 데이터가 vectordb에 넣어질때 사용할 임베딩 방식을 정합니다, 저희는 위에서 선언한 embeddings를 사용합니다.
            collection_name = 'india',                          # india라는 이름을 정해줌으로써 나중에 vector store 관리 가능 
            collection_metadata = {'hnsw:space': 'cosine'},     # cosine 말고 l2 가 default / collection_metadata를 통해 유사도 검색에 사용될 공간('hnsw:space')을 'cosine'으로 지정하여, 코사인 유사도를 사용
        )
        return vector_store

    def init_retriever(self):
        """ Retriever 초기화 """               

        # base retriever 3 
        retriever = self.vector_store.as_retriever(
            search_type="mmr",                                              # mmr 검색 방법으로 
            search_kwargs={'fetch_k': 10, "k": 5, 'lambda_mult': 0.4},      # 상위 10개의 관련 context에서 최종 5개를 추리고 'lambda_mult'는 관련성과 다양성 사이의 균형을 조정하는 파라메타 default 값이 0.5
            
        )
        
        return retriever
    
    # MultiQueryRetriever 생성
    def init_multi_query_retriever(self):
        """사용자의 질문을 여러 개의 유사 질문으로 재생성 """
        retriever_from_llm = MultiQueryRetriever.from_llm(
            llm=self.llm,
            retriever=self.retriever
        )

        return retriever_from_llm
    
    # SelfQueryRetriever 생성
    def init_self_query_retriever(self):
        """metadata를 이용해서 필터링해서 정보를 반환"""
        document_content_description = "Report or Explain"      # document_content_description: 문서 내용 설명
        retriever = SelfQueryRetriever.from_llm(    
            self.llm,
            self.vector_store,
            document_content_description,
            metadata_field_info,                                # metadata_field_info: 메타데이터 필드 정보
            verbose = True
        )
        
        return retriever
            

In [11]:
pipeline = Ragpipeline()

In [12]:
mq_retriever = pipeline.init_multi_query_retriever()
sq_retriever = pipeline.init_self_query_retriever()

In [13]:
query

'인도 통관 및 운송'

In [14]:
# Set logging for the queries
import logging

logging.basicConfig()
logging.getLogger('langchain.retrievers.multi_query').setLevel(logging.INFO)

result = mq_retriever.get_relevant_documents(query)


INFO:langchain.retrievers.multi_query:Generated queries: ['1. How to handle customs clearance and transportation in India?', '2. What are the procedures for customs clearance and shipping in India?', '3. Can you provide information on customs clearance and logistics in India?']


In [15]:

for i, doc in enumerate(result):
    print(i)
    print(f"문서 내용: {doc.page_content}") # 문서 내용 표시
    print(f'출처 : {doc.metadata}')
    print('---'*10)

0
문서 내용: 5. 통관 및 운송
 
 
가. 통관제도
  
 
통관 유형별 절차
 
1) 정식통관 
 
인도에서 일반적인 경우 통관에 소요되는 시간은 행정상 운송 수입의 경우 3~4 근무일, 항공 운송은 약 1~2 근무일이 소요된다. 일단 물
품이 입항하게 되면 보세구역장치에 적하되고, 이후 전자데이터 교환(EDI) 신고 또는 수작업신고 통해 수입신고를 하게 된다. 이후 인도
관세청의 수입요건, 관세 평가 등을 검사하는 Appraiser Section과 관련 서류 및 관세율을 재점검하는 Audit Section 과정을 거치게 되
며, 이를 통과하면 모든 서류에 서명 받게 된다. 이후 Challan No.라는 인도 관세 납부를 위한 번호를 부여받게 되고, 이를 근거로 관세를
완납하게 되면 물품을 반출할 수 있다. 또한, 정식통관의 경우 인도 세관은 일반적으로 화물 도착 후 7일 이후부터 체화료(Demurrage
Charge)를 부과하고, 컨테이너 운송의 경우 5일 이후부터 항공운송인 경우 3일 이
출처 : {'category': '정책', 'page': 0, 'source': '[정책][제약산업정보포털][2019.04.08]인도 통관 및 운송.pdf', 'year': 2019}
------------------------------
1
문서 내용:  
ㅇ Taewoong Logistics
 
 
ㅇ CJ Korea Express India Pvt. Ltd
 
 
ㅇ FIRST FLIGHT
 
 
ㅇ Gati Ltd.
 
 
 
 주소 No 75 beekeyen towers coral merchant street manady Chennai, Chennai, Tamil Nadu 600001
전화번호 +91)44-4343-2716
이메일 James@taewoong.in
홈페이지 http://www.e-tgl.com/eng/company02.do
비고 한국업체 
주소416, 4th Floor, Suncity Business Tower, Sector 54, Golf 

In [16]:
result = sq_retriever.get_relevant_documents(query)


In [17]:
for i, doc in enumerate(result):
    print(i)
    print(f"문서 내용: {doc.page_content}") # 문서 내용 표시
    print(f'출처 : {doc.metadata}')
    print("---" * 50)

0
문서 내용: 5. 통관 및 운송
 
 
가. 통관제도
  
 
통관 유형별 절차
 
1) 정식통관 
 
인도에서 일반적인 경우 통관에 소요되는 시간은 행정상 운송 수입의 경우 3~4 근무일, 항공 운송은 약 1~2 근무일이 소요된다. 일단 물
품이 입항하게 되면 보세구역장치에 적하되고, 이후 전자데이터 교환(EDI) 신고 또는 수작업신고 통해 수입신고를 하게 된다. 이후 인도
관세청의 수입요건, 관세 평가 등을 검사하는 Appraiser Section과 관련 서류 및 관세율을 재점검하는 Audit Section 과정을 거치게 되
며, 이를 통과하면 모든 서류에 서명 받게 된다. 이후 Challan No.라는 인도 관세 납부를 위한 번호를 부여받게 되고, 이를 근거로 관세를
완납하게 되면 물품을 반출할 수 있다. 또한, 정식통관의 경우 인도 세관은 일반적으로 화물 도착 후 7일 이후부터 체화료(Demurrage
Charge)를 부과하고, 컨테이너 운송의 경우 5일 이후부터 항공운송인 경우 3일 이
출처 : {'category': '정책', 'page': 0, 'source': '[정책][제약산업정보포털][2019.04.08]인도 통관 및 운송.pdf', 'year': 2019}
------------------------------------------------------------------------------------------------------------------------------------------------------
1
문서 내용: 도 증빙이 충분치 않다는 이유로 통관을 거부하는 사례도 자주 발생한다.  
 
  ㅇ 인도 지역별 세관관할: 인도의 세관 행정은 명목상 통일되어 있으나, 지역별 차이나 관할 세관 당국 간 소통 미비로 인하여 관세통관
상의 애로 해소가 발생하는 경우가 있다. 사전품목분류 심사제도의 경우에도 각 관할 세관별로 운영할 가능성이 높으며, 이에 한 관할지에
서 사전품목분류를 확정받았다 하더라도 다른 관할지에서는 

### 앙상블 검색기(Ensemble Retriever)

EnsembleRetriever는 여러 retriever를 입력으로 받아 get_relevant_documents() 메서드의 결과를 앙상블하고, Reciprocal Rank Fusion 알고리즘을 기반으로 결과를 재순위화합니다.

서로 다른 알고리즘의 장점을 활용함으로써, EnsembleRetriever는 단일 알고리즘보다 더 나은 성능을 달성할 수 있습니다.

가장 일반적인 패턴은 sparse retriever (예: BM25)와 dense retriever (예: embedding similarity)를 결합하는 것인데, 이는 두 retriever의 장점이 상호 보완적이기 때문입니다. 이를 "hybrid search" 라고도 합니다.

Sparse retriever는 키워드를 기반으로 관련 문서를 찾는 데 효과적이며, dense retriever는 의미적 유사성을 기반으로 관련 문서를 찾는 데 효과적입니다.

In [18]:
# Ensemble retriever
from langchain.retrievers import EnsembleRetriever
from langchain_community.retrievers import BM25Retriever
import pickle

In [19]:
class Ragpipeline:
    def __init__(self):
        # chatGPT API를 통해 llm 모델 로드
        self.llm = ChatOpenAI(
            model=config["llm_predictor"]["model_name"],  # chatgpt 모델 이름
            temperature=config["llm_predictor"]["temperature"],  # 창의성 0~1
        )
        
        # 초기화 리스트들 
        
        self.vector_store   = self.init_vectorDB()                  # 1. RAG가 접근할 vectorDB를 초기화합니다.
        self.retriever      = self.init_retriever()                 # 2. LLM이 질문에 대한 답변을 생성하기 전 질문에 관련된 컨텐츠 기반 답변을 생성하기 위해, 컨텐츠를 검색해 찾는 리트리버를 초기화 합니다. 
        self.bm25_retriever = self.init_bm25_retriever()
        self.ensemble_retriever = self.init_ensemble_retriever()
        self.mq_retriever   = self.init_multi_query_retriever()
        self.sq_retriever   = self.init_self_query_retriever()

    def init_vectorDB(self, persist_directory=config["chroma"]["persist_dir"]):
        """vectorDB 설정"""
        embeddings = OpenAIEmbeddings(model=config["embed_model"]["model_name"])  # VectorDB에 저장될 데이터를 임베딩할 모델을 선언합니다.
        vector_store = Chroma(
            persist_directory=persist_directory,                # 기존에 vectordb가 있으면 해당 위치의 vectordb를 load하고 없으면 새로 생성합니다.
            embedding_function=embeddings,                      # 새롭게 데이터가 vectordb에 넣어질때 사용할 임베딩 방식을 정합니다, 저희는 위에서 선언한 embeddings를 사용합니다.
            collection_name = 'india',                          # india라는 이름을 정해줌으로써 나중에 vector store 관리 가능 
            collection_metadata = {'hnsw:space': 'cosine'},     # cosine 말고 l2 가 default / collection_metadata를 통해 유사도 검색에 사용될 공간('hnsw:space')을 'cosine'으로 지정하여, 코사인 유사도를 사용
        )
        return vector_store

    def init_retriever(self):
        """ Retriever 초기화 """               

        # base retriever 3 
        retriever = self.vector_store.as_retriever(
            search_type="mmr",                                              # mmr 검색 방법으로 
            search_kwargs={'fetch_k': 10, "k": 5, 'lambda_mult': 0.4},      # 상위 10개의 관련 context에서 최종 5개를 추리고 'lambda_mult'는 관련성과 다양성 사이의 균형을 조정하는 파라메타 default 값이 0.5
            
        )
        
        return retriever
    
    def init_bm25_retriever(self):

        all_docs = pickle.load(open(config["pkl_path"], 'rb'))              # pkl파일에 내용들에서 bm25

        bm25_retriever = BM25Retriever.from_documents(all_docs)
        bm25_retriever.k = 1                                                # BM25Retriever의 검색 결과 개수를 1로 설정합니다.

        return bm25_retriever
    
    def init_ensemble_retriever(self):

        bm25_retriever = self.bm25_retriever
        chroma_retriever = self.retriever

        ensemble_retriever = EnsembleRetriever(
            retrievers=[bm25_retriever, chroma_retriever],
            weights=[0.4, 0.6],
            search_type=config["ensemble_search_type"],  # mmr
        )

        return ensemble_retriever
    
    # MultiQueryRetriever 생성
    def init_multi_query_retriever(self):
        """사용자의 질문을 여러 개의 유사 질문으로 재생성 """
        retriever_from_llm = MultiQueryRetriever.from_llm(
            llm=self.llm,
            retriever=self.retriever
        )

        return retriever_from_llm
    
    # SelfQueryRetriever 생성
    def init_self_query_retriever(self):
        """metadata를 이용해서 필터링해서 정보를 반환"""
        document_content_description = "Report or Explain"      # document_content_description: 문서 내용 설명
        retriever = SelfQueryRetriever.from_llm(    
            self.llm,
            self.vector_store,
            document_content_description,
            metadata_field_info,                                # metadata_field_info: 메타데이터 필드 정보
            verbose = True
        )
        
        return retriever
            

In [20]:
pipeline = Ragpipeline()

In [21]:
ensemble_retriever = pipeline.init_ensemble_retriever()

In [22]:
query = '인도 통관 및 운송'   # 질문할 문장

In [23]:
result = ensemble_retriever.get_relevant_documents(query)

for i, doc in enumerate(result):
    print(i)
    print(f"문서 내용: {doc.page_content}") # 문서 내용 표시
    print(f'출처 : {doc.metadata}')
    print("---" * 50)

0
문서 내용: 5. 통관 및 운송
 
 
가. 통관제도
  
 
통관 유형별 절차
 
1) 정식통관 
 
인도에서 일반적인 경우 통관에 소요되는 시간은 행정상 운송 수입의 경우 3~4 근무일, 항공 운송은 약 1~2 근무일이 소요된다. 일단 물
품이 입항하게 되면 보세구역장치에 적하되고, 이후 전자데이터 교환(EDI) 신고 또는 수작업신고 통해 수입신고를 하게 된다. 이후 인도
관세청의 수입요건, 관세 평가 등을 검사하는 Appraiser Section과 관련 서류 및 관세율을 재점검하는 Audit Section 과정을 거치게 되
며, 이를 통과하면 모든 서류에 서명 받게 된다. 이후 Challan No.라는 인도 관세 납부를 위한 번호를 부여받게 되고, 이를 근거로 관세를
완납하게 되면 물품을 반출할 수 있다. 또한, 정식통관의 경우 인도 세관은 일반적으로 화물 도착 후 7일 이후부터 체화료(Demurrage
Charge)를 부과하고, 컨테이너 운송의 경우 5일 이후부터 항공운송인 경우 3일 이
출처 : {'category': '정책', 'page': 0, 'source': '[정책][제약산업정보포털][2019.04.08]인도 통관 및 운송.pdf', 'year': 2019}
------------------------------------------------------------------------------------------------------------------------------------------------------
1
문서 내용: 도 증빙이 충분치 않다는 이유로 통관을 거부하는 사례도 자주 발생한다.  
 
  ㅇ 인도 지역별 세관관할: 인도의 세관 행정은 명목상 통일되어 있으나, 지역별 차이나 관할 세관 당국 간 소통 미비로 인하여 관세통관
상의 애로 해소가 발생하는 경우가 있다. 사전품목분류 심사제도의 경우에도 각 관할 세관별로 운영할 가능성이 높으며, 이에 한 관할지에
서 사전품목분류를 확정받았다 하더라도 다른 관할지에서는 

### Multi-Query Retriever와 Ensemble Retriever를 함께 사용하면 어떻게 될까요?

In [24]:
class Ragpipeline:
    def __init__(self):
        # chatGPT API를 통해 llm 모델 로드
        self.llm = ChatOpenAI(
            model=config["llm_predictor"]["model_name"],  # chatgpt 모델 이름
            temperature=config["llm_predictor"]["temperature"],  # 창의성 0~1
        )
        
        # 초기화 리스트들 
        self.vector_store   = self.init_vectorDB()                  # 1. RAG가 접근할 vectorDB를 초기화합니다.
        self.retriever      = self.init_retriever()                 # 2. LLM이 질문에 대한 답변을 생성하기 전 질문에 관련된 컨텐츠 기반 답변을 생성하기 위해, 컨텐츠를 검색해 찾는 리트리버를 초기화 합니다. 
        self.bm25_retriever = self.init_bm25_retriever()
        self.ensemble_retriever = self.init_ensemble_retriever()
        self.mq_retriever   = self.init_multi_query_retriever()
        self.sq_retriever   = self.init_self_query_retriever()
        
        self.mq_ensemble_retriever = self.init_mq_ensemble_retriever()

    def init_vectorDB(self, persist_directory=config["chroma"]["persist_dir"]):
        """vectorDB 설정"""
        embeddings = OpenAIEmbeddings(model=config["embed_model"]["model_name"])  # VectorDB에 저장될 데이터를 임베딩할 모델을 선언합니다.
        vector_store = Chroma(
            persist_directory=persist_directory,  # 기존에 vectordb가 있으면 해당 위치의 vectordb를 load하고 없으면 새로 생성합니다.
            embedding_function=embeddings,                      # 새롭게 데이터가 vectordb에 넣어질때 사용할 임베딩 방식을 정합니다, 저희는 위에서 선언한 embeddings를 사용합니다.
            collection_name = 'india',                          # india라는 이름을 정해줌으로써 나중에 vector store 관리 가능 
            collection_metadata = {'hnsw:space': 'cosine'},     # cosine 말고 l2 가 default / collection_metadata를 통해 유사도 검색에 사용될 공간('hnsw:space')을 'cosine'으로 지정하여, 코사인 유사도를 사용
        )
        return vector_store

    def init_retriever(self):
        """ Retriever 초기화 """               

        # base retriever 3 
        retriever = self.vector_store.as_retriever(
            search_type="mmr",                                              # mmr 검색 방법으로 
            search_kwargs={'fetch_k': 10, "k": 5, 'lambda_mult': 0.4},      # 상위 10개의 관련 context에서 최종 5개를 추리고 'lambda_mult'는 관련성과 다양성 사이의 균형을 조정하는 파라메타 default 값이 0.5
            
        )
        
        return retriever
    
    def init_bm25_retriever(self):

        all_docs = pickle.load(open(config["pkl_path"], 'rb'))

        bm25_retriever = BM25Retriever.from_documents(all_docs)
        bm25_retriever.k = 1                                            # BM25Retriever의 검색 결과 개수를 1로 설정합니다.

        return bm25_retriever
    
    def init_ensemble_retriever(self):

        bm25_retriever = self.bm25_retriever
        chroma_retriever = self.retriever

        ensemble_retriever = EnsembleRetriever(
            retrievers=[bm25_retriever, chroma_retriever],
            weights=[0.4, 0.6],
            search_type=config["ensemble_search_type"],  # mmr
        )

        return ensemble_retriever
    
    # MultiQueryRetriever 생성
    def init_multi_query_retriever(self):
        """사용자의 질문을 여러 개의 유사 질문으로 재생성 """
        retriever_from_llm = MultiQueryRetriever.from_llm(
            llm=self.llm,
            retriever=self.retriever
        )

        return retriever_from_llm
    
    # SelfQueryRetriever 생성
    def init_self_query_retriever(self):
        """metadata를 이용해서 필터링해서 정보를 반환"""
        document_content_description = "Report or Explain"      # document_content_description: 문서 내용 설명
        retriever = SelfQueryRetriever.from_llm(    
            self.llm,
            self.vector_store,
            document_content_description,
            metadata_field_info,                                # metadata_field_info: 메타데이터 필드 정보
            verbose = True
        )
        
        return retriever
    
    # 멀티쿼리 - 앙승블
    def init_mq_ensemble_retriever(self):

        mq_ensemble_retriever = MultiQueryRetriever.from_llm(
            llm=self.llm,
            retriever=self.ensemble_retriever
        )

        return mq_ensemble_retriever
            

In [25]:
pipeline = Ragpipeline()

In [26]:
mq_ensemble_retriever = pipeline.init_mq_ensemble_retriever()

In [27]:
query = '인도 통관 및 운송'   # 질문할 문장

In [28]:
result = mq_ensemble_retriever.get_relevant_documents(query)

for i, doc in enumerate(result):
    print(i)
    print(f"문서 내용: {doc.page_content}") # 문서 내용 표시
    print(f'출처 : {doc.metadata}')
    print("---" * 50)

INFO:langchain.retrievers.multi_query:Generated queries: ['1. What are the customs clearance and transportation processes in India?', '2. Can you provide information on customs clearance and shipping in India?', '3. How does customs clearance and transportation work in India?']


0
문서 내용: 5. 통관 및 운송
 
 
가. 통관제도
  
 
통관 유형별 절차
 
1) 정식통관 
 
인도에서 일반적인 경우 통관에 소요되는 시간은 행정상 운송 수입의 경우 3~4 근무일, 항공 운송은 약 1~2 근무일이 소요된다. 일단 물
품이 입항하게 되면 보세구역장치에 적하되고, 이후 전자데이터 교환(EDI) 신고 또는 수작업신고 통해 수입신고를 하게 된다. 이후 인도
관세청의 수입요건, 관세 평가 등을 검사하는 Appraiser Section과 관련 서류 및 관세율을 재점검하는 Audit Section 과정을 거치게 되
며, 이를 통과하면 모든 서류에 서명 받게 된다. 이후 Challan No.라는 인도 관세 납부를 위한 번호를 부여받게 되고, 이를 근거로 관세를
완납하게 되면 물품을 반출할 수 있다. 또한, 정식통관의 경우 인도 세관은 일반적으로 화물 도착 후 7일 이후부터 체화료(Demurrage
Charge)를 부과하고, 컨테이너 운송의 경우 5일 이후부터 항공운송인 경우 3일 이
출처 : {'category': '정책', 'page': 0, 'source': '[정책][제약산업정보포털][2019.04.08]인도 통관 및 운송.pdf', 'year': 2019}
------------------------------------------------------------------------------------------------------------------------------------------------------
1
문서 내용: 11
대사관에서 알려드리는 인도 경제 소식지[표-6]   인디안 트레이트 포탈 메인 화면
인도 정부에서도 “인디안 트레이드 포탈(https://www.indiantrade portal. 
in/”)에서 품목분류 검색 정보를 제공하고 있지만, 우리나라보다 정보가 늦고 수시로 
세율이 변경되기 때문에 정확한 정보를 찾기에는 한계가 있습니다. 가능한 책자를 
찾아보거나 전문가를 통하여 상담을 받고 정확한 분류